### Hyperspy notebook with parallel code to generate video from in-situ TEM mode
#### by Joseph Vas NIE, Singapore joseph.vas@nie.edu.sg

usual imports for data processing


In [1]:
%matplotlib notebook
import glob
import matplotlib.pyplot as plt 
from os import listdir
import matplotlib.animation as animation
import numpy as np
from scipy import ndimage
import os

In [2]:
import hyperspy.api as hs

/root/anaconda3/lib/python3.8/site-packages/pyUSID/viz/__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


Step 2: import the function which needs to run in parallel 

In [3]:
import test
from multiprocessing import Pool

initializing the variables

In [4]:
i=0
filename = '-4mA_100kHz_1s_5pr_Hour_00_Minute_00_Second_00_'
path0 = '../../Demo_data/Minute_'
path1 = str(i).zfill(2)+'/Second_'

In [5]:
def concatenate():
    stringa = "ffmpeg -i \"concat:"
    elenco_video = glob.glob("*.mp4")
    elenco_file_temp = []
    for f in elenco_video:
        file = "temp" + str(elenco_video.index(f) + 1) + ".ts"
        os.system("ffmpeg -i " + f + " -c copy -bsf:v h264_mp4toannexb -f mpegts " + file)
        elenco_file_temp.append(file)
    print(elenco_file_temp)
    for f in elenco_file_temp:
        stringa += f
        if elenco_file_temp.index(f) != len(elenco_file_temp)-1:
            stringa += "|"
        else:
            stringa += "\" -c copy  -bsf:a aac_adtstoasc output.mp4"
    #rint(stringa)
    os.system(stringa)

initializing the codec for creating the video

In [14]:
%%time
k_max = len(listdir(path0+str(0).zfill(2)))
if __name__ == '__main__':
    num_processors = 8
    p = Pool(processes = num_processors)
    output = p.map(test.parallel_save,[i for i in range(0,k_max)])
    #print(output)
#test.parallel_save(path,file_name)
concatenate()

Hour_00/Minute_00/Second_01/Frame04
Hour_00/Minute_00/Second_01/Frame05
Hour_00/Minute_00/Second_01/Frame06
Hour_00/Minute_00/Second_01/Frame07
Hour_00/Minute_00/Second_01/Frame08
Hour_00/Minute_00/Second_01/Frame09
Hour_00/Minute_00/Second_01/Frame10
Hour_00/Minute_00/Second_01/Frame11
Hour_00/Minute_00/Second_01/Frame12
Hour_00/Minute_00/Second_01/Frame13
Hour_00/Minute_00/Second_01/Frame14
Hour_00/Minute_00/Second_01/Frame15
Hour_00/Minute_00/Second_01/Frame16
Hour_00/Minute_00/Second_01/Frame17
Hour_00/Minute_00/Second_01/Frame18
Hour_00/Minute_00/Second_01/Frame19
Hour_00/Minute_00/Second_01/Frame20
Hour_00/Minute_00/Second_01/Frame21
Hour_00/Minute_00/Second_01/Frame22
Hour_00/Minute_00/Second_01/Frame23
Hour_00/Minute_00/Second_01/Frame24
['temp1.ts', 'temp2.ts']
CPU times: user 35.6 ms, sys: 85 ms, total: 121 ms
Wall time: 1min 31s


concatinating individual videos for final video     

In [11]:
def parallel_save(i):
    vmin_mul = 0
    vmax_mul = 1
    fig, axes = plt.subplots(nrows=1, ncols=1)
    filename = '-4mA_100kHz_1s_5pr_Hour_00_Minute_00_'+'Second_'+str(i).zfill(2)+'_'
    FFMpegWriter = animation.writers['ffmpeg']
    metadata = dict(title=path, artist='J.Vas')
    writer = FFMpegWriter(fps=25, metadata=metadata, bitrate=200)
    k_max = len(listdir('Hour_00/Minute_00/Second_'+str(i).zfill(2)))
    targetfile = 'second'+str(i).zfill(2)+'.mp4'
    with writer.saving(fig, targetfile, dpi=200):
        for k in range(0, k_max):
            TEM = hs.load(path + str(i).zfill(2)+'/'+filename+'Frame_00' + str(k).zfill(2) + '.dm4')
            if np.max(TEM.data) - np.min(TEM.data) > 0:
                TEM = ndimage.rotate(TEM, 250, reshape=False)
                TEM = TEM[300:3700, 750:3100]
                im = axes.imshow(ndimage.gaussian_filter(TEM.data, 6), cmap='gray', \
                                         vmin=vmin_mul * np.mean(TEM.data), vmax=vmax_mul * np.mean(TEM.data))
                print('Hour_00/Minute_00' + '/' + 'Second_' + str(i).zfill(2) + '/Frame' + str(k).zfill(2))
                axes.set_yticklabels([])
                axes.set_xticklabels([])
                plt.ioff()
                plt.xticks([])
                plt.yticks([])
                plt.hlines(2900, 325, 665)
                plt.text(328, 2800, '1 ' + u"\u03bcm")
                writer.grab_frame()

In [12]:
concatenate()

['temp1.ts']
